In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_seq': 40,
    'min_freq': 0,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [3]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, config)

In [4]:
# for i, sentence in enumerate(AttractiveData.test_data):
#     if i == 3:
#         print(vars(AttractiveData.train_data[i]), vars(sentence))

In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [6]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

38

## Start to train

In [7]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 5
config['hidden_dim'] = 30
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 300
config['lr'] = {
    'encoder': 1e-5,
    'embedding': 5e-6,
    'linear': 1e-5
}
config['num_layers'] = 1
config['kernel_size'] = 3
config['dropout'] = 0.1
config['train_len'] = AttractiveData.train_len
config['val_len'] = AttractiveData.val_len
config['test_len'] = AttractiveData.test_len

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12699, 300])


In [8]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, AttractiveData.valloader, pretrained_embeddings)

In [9]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(12699, 300, padding_idx=1)
   )
   (bigramcnn): Sequential(
     (0): Conv1d(300, 220, kernel_size=(2,), stride=(1,), padding=(1,))
     (1): ReLU()
     (2): Conv1d(220, 150, kernel_size=(2,), stride=(1,), padding=(1,))
     (3): ReLU()
     (4): Conv1d(150, 100, kernel_size=(2,), stride=(1,), padding=(1,))
     (5): ReLU()
   )
   (trigramcnn): Sequential(
     (0): Conv1d(300, 220, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
     (2): Conv1d(220, 150, kernel_size=(3,), stride=(1,), padding=(1,))
     (3): ReLU()
     (4): Conv1d(150, 100, kernel_size=(3,), stride=(1,), padding=(1,))
     (5): ReLU()
   )
   (encoder_bigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (encoder_trigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (linear): Sequential(
     (0): Linear(in_features=240, out_features=30, bias=True)
     (1): ReLU()
     (2)

In [10]:
AttractiveTrainer.train()

Epoch:   0%|          | 1/300 [00:00<02:45,  1.81it/s]
EP_train | train loss: 0.5322980001860974 | val loss: 0.5427955365648457 |
====



KeyboardInterrupt: 

## for classification, not better

## Below is testing

In [8]:
from attractivenet import AttractiveNet
PATH = './model/CNN_LSTM_20201101-230925_0.4092.170'
# load_model = TransformerModel(config).to(AttractiveData.device)
load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(12699, 300, padding_idx=1)
  )
  (bigramcnn): Sequential(
    (0): Conv1d(300, 220, kernel_size=(2,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(220, 150, kernel_size=(2,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): Conv1d(150, 100, kernel_size=(2,), stride=(1,), padding=(1,))
    (5): ReLU()
  )
  (trigramcnn): Sequential(
    (0): Conv1d(300, 220, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(220, 150, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): Conv1d(150, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): ReLU()
  )
  (encoder_bigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (encoder_trigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=240, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, o

In [9]:
def predict_attractive(sentence, category, phase):
    # if len(sentence) < config['max_seq']:
    #     sentence += ['0'] * (config['max_seq'] - len(sentence))
    # else:
    #     sentence = sentence[:config['max_seq']]

    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(0)
    # print(tensor_sentence.shape)

    prediction = load_model(tensor_sentence, tensor_category, phase=phase)

    # after_decimal = prediction % 1
    # possible_list = [0.0, 0.3333333333333333, 0.6666666666666665, 0.5, 1.0]
    # closet_idx = None
    # closet_distance = 1
    # for i in range(len(possible_list)):
    #     if abs(after_decimal - possible_list[i]) <= closet_distance:
    #         closet_idx = i
    #         closet_distance = abs(after_decimal - possible_list[i])
    # prediction = (prediction // 1) + possible_list[closet_idx]
    
    return prediction

In [10]:
# train mean = 3.15, test mean = 2.8
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        prediction = predict_attractive(sentence.Headline, sentence.Category, 'test')
        predict_list.append(prediction.item())
        # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

## Below just for fun guess

In [11]:
import statistics
from sklearn.metrics import mean_squared_error

In [12]:
# train mean = 3.15, test mean = 2.8
train_list = []
for i, sentence in enumerate(AttractiveData.train_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category, 'train')
    train_list.append(prediction.item())
    # train_list.append(prediction.item())
# print(train_list)
mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

(0.3805509350909898, 3.2406243774236416, 0.31552978540363424)

In [13]:
train_list[0:5], pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list()[0:5]

([3.208599328994751,
  2.7554450035095215,
  2.9379777908325195,
  3.351276159286499,
  3.4500410556793213],
 [4.0, 2.333333333333333, 4.5, 3.333333333333333, 4.0])

In [14]:
a = AttractiveData.df_train['Label'].to_list()
statistics.mean(a), statistics.stdev(a)

(3.150408496732026, 0.729501519321601)

In [15]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.7519895085154125, 0.31422964573860457)

In [16]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.03938576045220719, 2.7156126782757597, 0.29355123275379763)

In [14]:
# LSTM my best
# statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8167915543795683, 0.14611407210842048)

In [21]:
a = pd.read_csv('LSTM_base.csv').Label.to_list()
mean_squared_error(baseline_list, a), statistics.mean(a), statistics.stdev(a)

(0.1347375515605904, 2.8379913731293533, 0.1903582104725371)

In [14]:
AttractiveData.TEXT.vocab.itos[2]

'the'

In [26]:
AttractiveData.LABEL.vocab.freqs

Counter({'4.0': 226,
         '2.333333333333333': 194,
         '4.5': 43,
         '3.333333333333333': 313,
         '3.6666666666666665': 260,
         '2.6666666666666665': 281,
         '2.0': 135,
         '2.5': 36,
         '1.6666666666666667': 28,
         '3.0': 354,
         '4.333333333333333': 82,
         '4.666666666666667': 29,
         '1.5': 16,
         '3.5': 22,
         '1.3333333333333333': 4,
         '1.0': 5,
         '5.0': 12})

In [29]:
for i in range(1, 4):
    print(1 / i)
# 0, 0.33, 0.5, 0.66, 1

1.0
0.5
0.3333333333333333


In [31]:
0.3333333333333333 % 1

0.3333333333333333